In [1]:
import os
os.chdir("/data/yosef2/users/chenling/HarmonizationSCANVI")
import sys
sys.path.append("/data/yosef2/users/chenling/HarmonizationSCANVI")
save_path = '../MarrowTM'

In [2]:
from scvi.inference import UnsupervisedTrainer
from scvi.models.vae import VAE
import torch

In [3]:
use_cuda = True
from scvi.dataset.dataset import GeneExpressionDataset
from scvi.harmonization.utils_chenling import run_model
from scvi.harmonization.utils_chenling import entropy_batch_mixing
from scvi.metrics.clustering import select_indices_evenly
from scvi.dataset.dataset import SubsetGenes

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns
import numpy as np
from umap import UMAP


In [4]:
plotname='MarrowTM'

In [5]:
from scvi.dataset.muris_tabula import TabulaMuris
dataset2 = TabulaMuris('droplet', save_path='/data/yosef2/scratch/chenling/scanvi_data/')
dataset1 = TabulaMuris('facs', save_path='/data/yosef2/scratch/chenling/scanvi_data/')
dataset1.subsample_genes(dataset1.nb_genes)
dataset2.subsample_genes(dataset2.nb_genes)

gene_dataset = GeneExpressionDataset.concat_datasets(dataset1, dataset2)
seurat_latent, batch_indices, labels, keys, stats = run_model('readSeurat', gene_dataset, dataset1, dataset2,
                                                       filename=plotname)
dataset1, dataset2, gene_dataset = SubsetGenes(dataset1, dataset2, gene_dataset, plotname)

File /data/yosef2/scratch/chenling/scanvi_data/TM_droplet_metadata.csv already downloaded
File /data/yosef2/scratch/chenling/scanvi_data/TM_facs_metadata.csv already downloaded
File /data/yosef2/scratch/chenling/scanvi_data/TM_droplet_mat.h5ad already downloaded
File /data/yosef2/scratch/chenling/scanvi_data/TM_facs_mat.h5ad already downloaded
File /data/yosef2/scratch/chenling/scanvi_data/TM_droplet_metadata.csv already downloaded
File /data/yosef2/scratch/chenling/scanvi_data/TM_facs_metadata.csv already downloaded
File /data/yosef2/scratch/chenling/scanvi_data/TM_droplet_mat.h5ad already downloaded
File /data/yosef2/scratch/chenling/scanvi_data/TM_facs_mat.h5ad already downloaded
True
Downsampling from 20508 to 17802 genes
Downsampling from 5351 to 5351 cells
Downsampling from 20508 to 14687 genes
Downsampling from 4112 to 4112 cells
Keeping 14590 genes
Downsampling from 17802 to 1804 genes
Downsampling from 5351 to 5351 cells
Downsampling from 14687 to 1804 genes
Downsampling from 

In [6]:
seurat_latent, batch_indices, labels, keys, stats = run_model('readSeurat', gene_dataset, dataset1, dataset2,
                                                       filename=plotname)


In [7]:
from sklearn.neighbors import KNeighborsClassifier
def transfer_nn_labels(latent_array, labels_array, batch_indices_array):
    # Transfer labels from batch 0 to batch 1 using scVI
    latent_labelled = latent_array[batch_indices_array.ravel() == 0, :]
    labels_labelled = labels_array[batch_indices_array.ravel() == 0]
    neigh = KNeighborsClassifier(n_neighbors=10)
    neigh = neigh.fit(latent_labelled, labels_labelled)
    return neigh.predict(latent_array)


In [9]:
from time import time

In [10]:
start = time()
transfer_nn_labels(seurat_latent, labels, batch_indices)
print(time()-start)

0.5551154613494873


In [12]:
seurat_latent.shape

(9463, 10)

In [43]:
from scvi.dataset import GeneExpressionDataset
from scvi.models import VAE
from scvi.inference import UnsupervisedTrainer, AlternateSemiSupervisedTrainer
from scvi.inference.posterior import get_IS_bayes_factors
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from scvi.models.scanvi import SCANVI

def transfer_nn_labels(latent_array, labels_array, batch_indices_array):
    # Transfer labels from batch 0 to batch 1 using scVI
    latent_labelled = latent_array[batch_indices_array.ravel() == 0, :]
    labels_labelled = labels_array[batch_indices_array.ravel() == 0]
    neigh = KNeighborsClassifier(n_neighbors=10)
    neigh = neigh.fit(latent_labelled, labels_labelled)
    return neigh.predict(latent_array)


def get_bayes_factor_scvi(cset_a, cset_b, sampling_n, cells_sampled, use_is, force_batch=None):
    subset_a = np.random.choice(cset_a, cells_sampled)
    subset_b = np.random.choice(cset_b, cells_sampled)
    posterior_a = trainer.create_posterior(trainer.model, gene_dataset,
                                           indices=subset_a)
    posterior_b = trainer.create_posterior(trainer.model, gene_dataset,
                                           indices=subset_b)
    px_scale_a, log_ratios_a, labels_a = posterior_a.differential_expression_stats(M_sampling=sampling_n,
                                                                                   force_batch=force_batch)
    px_scale_b, log_ratios_b, labels_b = posterior_b.differential_expression_stats(M_sampling=sampling_n,
                                                                                   force_batch=force_batch)
    px_scale = np.concatenate((px_scale_a, px_scale_b), axis=1)
    log_ratios = np.concatenate((log_ratios_a, log_ratios_b), axis=1)
    labels_de = np.concatenate((0 * labels_a, 0 * labels_b + 1), axis=0)
    return get_IS_bayes_factors(px_scale, log_ratios, labels_de, 0,
                                other_cell_idx=1,
                                importance_sampling=use_is, permutation=False)


def eval_bayes_factor(log_fold_change, bayes_f):
    """
    :param log_fold_change: groundtruth
    :param bayes_f: non-log Bayes Factor
    :return:
    """
    bayes_f = np.log(bayes_f + 1e-8) - np.log(1 - bayes_f + 1e-8)
    auc_1 = roc_auc_score(np.abs(log_fold_change) >= 0.6, np.abs(bayes_f))
    auc_2 = roc_auc_score(np.abs(log_fold_change) >= 0.8, np.abs(bayes_f))
    spear = spearmanr(bayes_f, log_fold_change)[0]
    kend = kendalltau(bayes_f, log_fold_change)[0]
    return auc_1, auc_2, spear, kend



In [45]:
import pandas as pd

In [48]:
save_path = "../symsim_scVI/symsim_result/DE/"
label_array = pd.read_csv(os.path.join(save_path, "DE.cell_meta.csv"),
                          sep=",", index_col=0)["pop"].values
batch_array = pd.read_csv(os.path.join(save_path, "DE.batchid.csv"),
                          sep=",", index_col=0)["x"].values
# Renumerate the batches to be between 0 and N-batches
batch_array -= 1
batch_array = batch_array[:, np.newaxis]

count_matrix = pd.read_csv(os.path.join(save_path, "DE.obsv.2.csv"),
                           sep=",", index_col=0).T

gene_names = np.array(count_matrix.columns, dtype=str)

dataset1 = GeneExpressionDataset(*GeneExpressionDataset.get_attributes_from_matrix(
    count_matrix.values, labels=label_array,batch_indices=batch_array),
    gene_names=gene_names, cell_types=np.unique(label_array))

dataset1.update_cells(batch_array.ravel()==0)

count_matrix = pd.read_csv(os.path.join(save_path, "DE.obsv.4.csv"),
                           sep=",", index_col=0).T

dataset2 = GeneExpressionDataset(*GeneExpressionDataset.get_attributes_from_matrix(
    count_matrix.values, labels=label_array,batch_indices=batch_array),
    gene_names=gene_names, cell_types=np.unique(label_array))

dataset2.update_cells(batch_array.ravel()==1)

gene_dataset = GeneExpressionDataset.concat_datasets(dataset1, dataset2)
labels = [int(gene_dataset.cell_types[i])-1 for i in gene_dataset.labels.ravel()]
gene_dataset.labels = np.asarray(labels).reshape(len(labels),1)
gene_dataset.cell_types = dataset2.cell_types


Downsampling from 20000 to 9888 cells
Downsampling from 20000 to 10112 cells
Keeping 5000 genes


In [51]:
vae = VAE(gene_dataset.nb_genes, n_batch=gene_dataset.n_batches, reconstruction_loss="zinb", n_latent=10)
trainer = UnsupervisedTrainer(vae,
                              gene_dataset,
                              train_size=0.75,
                              use_cuda=True,
                              frequency=5, kl=1)

n_epochs = 1
trainer.train(n_epochs=n_epochs, lr=0.001)
start=time()
full = trainer.create_posterior(trainer.model, gene_dataset, indices=np.arange(len(gene_dataset)))
latent, batch_indices, _ = full.sequential().get_latent()
print('time to get scvi latent space %.2f'%(time()-start))


training: 100%|██████████| 1/1 [00:03<00:00,  3.28s/it]
time to get scvi latent space 0.65


In [53]:

start=time()
print("Transferring labels from scVI")
scVI_labels = transfer_nn_labels(np.asarray(latent), np.asarray(labels), np.asarray(batch_indices))
print('time to transfer labels using knn %.2f'%(time()-start))


Transferring labels from scVI
time to transfer labels using knn 0.88


In [54]:

# train scANVI
print("Training scANVI")
scanvi = SCANVI(gene_dataset.nb_genes, gene_dataset.n_batches, gene_dataset.n_labels, n_latent=10)
scanvi.load_state_dict(trainer.model.state_dict(), strict=False)
trainer_scanvi = AlternateSemiSupervisedTrainer(scanvi, gene_dataset,
                                                n_epochs_classifier=5, lr_classification=5 * 1e-3, kl=1)
labelled = np.where(gene_dataset.batch_indices == 0)[0]
np.random.shuffle(labelled)
unlabelled = np.where(gene_dataset.batch_indices == 1)[0]
np.random.shuffle(unlabelled)
trainer_scanvi.labelled_set = trainer_scanvi.create_posterior(indices=labelled)
trainer_scanvi.unlabelled_set = trainer_scanvi.create_posterior(indices=unlabelled)

trainer_scanvi.train(n_epochs=1)


Training scANVI
training: 100%|██████████| 1/1 [00:07<00:00,  7.56s/it]


In [55]:

start=time()
print("Transferring labels from scVI")
scanvi_labels = trainer_scanvi.full_dataset.sequential().compute_predictions()[1]
print('time to transfer labels using scanvi %.2f'%(time()-start))


Transferring labels from scVI
time to transfer labels using scanvi 0.68


In [58]:
labels=np.asarray(labels)

In [61]:
start=time()
set_a = np.where(
    np.logical_and(labels == 0, gene_dataset.batch_indices.ravel() == 0))[0]
set_b = np.where(
    np.logical_and(labels == 1, gene_dataset.batch_indices.ravel() == 0))[0]
print(len(set_a),len(set_b))
bayes_A = get_bayes_factor_scvi(set_a, set_b, 10, 1000, use_is=False)
print('time to compute bayes factor %.2f'%(time()-start))


2012 2014
time to compute bayes factor 3.62
